# Getting Started with BentoML

BentoML is a python framework for building, shipping and running machine learning services. It provides high-level APIs for defining an ML service and packaging its artifacts, source code, dependencies, and configurations into a production-system-friendly format that is ready for deployment.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=nb&ea=open&el=official-example&dt=bentoml-quick-start-guide)

In [ ]:
# Install BentoML
!pip install -I bentoml

# Install scikit-learn, we will use a sklean model as an example
!pip install pandas sklearn

Let's get started with a simple scikit-learn model as an example:

In [ ]:
from sklearn import svm
from sklearn import datasets

clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)

## Define ML Service with BentoML

To package this model with BentoML, you don't need to change anything in your training code. Following your training workflow, create a new BentoML Service by subclassing it:

In [ ]:
%%writefile iris_classifier.py
from bentoml import BentoService, api, env, artifacts
from bentoml.artifact import PickleArtifact
from bentoml.handlers import DataframeHandler

# You can also import your own python module here and BentoML will automatically
# figure out the dependency chain and package all those python modules

@artifacts([PickleArtifact('model')])
@env(conda_pip_dependencies=["scikit-learn"])
class IrisClassifier(BentoService):

    @api(DataframeHandler)
    def predict(self, df):
        # arbitrary preprocessing or feature fetching code can be placed here 
        return self.artifacts.model.predict(df)

The `@artifacts` decorator here tells BentoML what artifacts are required when 
packaging this BentoService. Besides `PickleArtifact`, BentoML also provides
`TfKerasModelArtifact`, `PytorchModelArtifact`, and `TfSavedModelArtifact` etc.

`@env` is designed for specifying the desired system environment in order for this
BentoService to load. Other ways you can use this decorator:

* If you already have a requirement.txt file listing all python libraries you
need:
```python
@env(requirement_txt='../myproject/requirement.txt')
```

* Or if you are running this code in a Conda environment that matches the
desired production environment:
```python
@env(with_current_conda_env=True)
```

Lastly `@api` adds an entry point for accessing this BentoService. Each
`api` will be translated into a REST endpoint when [deploying as API
server](#serving-via-rest-api), or a CLI command when [running as a CLI
tool](#use-as-cli-tool).

Each API also requires a `Handler` for defining the expected input format. In
this case, `DataframeHandler` will transform either an HTTP request or CLI
command arguments into a pandas Dataframe and pass it down to the user defined
API function. BentoML also supports `JsonHandler`, `ImageHandler` and
`TensorHandler`.


## Save BentoML service archive

Pack your custom BentoML Service with the trained model:

In [ ]:
# 1) import the custom BentoService defined above
from iris_classifier import IrisClassifier

# 2) `pack` it with required artifacts
svc = IrisClassifier.pack(model=clf)

# 3) save packed BentoService as archive
saved_path = svc.save('/tmp/bentoml_archive')

# archive will be saved to:
print(saved_path)

_That's it._ You've just created your first BentoML servie archive. It's a directory
containing all the source code, data and configurations files required to load
and run a BentoService. You will also find three 'magic' files generated
within the archive directory:

* `bentoml.yml` - a YAML file containing all metadata related to this
  BentoArchive
* `Dockerfile` - for building a Docker Image exposing this BentoService as REST
  API endpoint
* `setup.py` - the config file that makes a BentoArchive 'pip' installable

In [ ]:
# For demo purpurse, copy generated model to ./model folder
import shutil
shutil.rmtree('./model', ignore_errors=True)
shutil.copytree(saved_path, './model')

## Serving via REST API

For exposing your model as a HTTP API endpoint, you can simply use the `bentoml
serve` command:

In [ ]:
!bentoml serve ./model

#### Send prediction request to REST API server

*Run the following command in terminal to make a HTTP request to the API server*
```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[[5.1, 3.5, 1.4, 0.2]]' \
localhost:5000/predict
```

Note you must ensure the pip and conda dependencies are available in your python
environment when using `bentoml serve` command. More commonly we recommend using
BentoML API server with Docker:

## Run REST API server with Docker

You can build a Docker Image for running API server hosting your BentoML archive
by using the archive folder as docker build context:

__`docker` is note available on Google Colab__

In [ ]:
!cd ./model && docker build -t iris-classifier .

Next, you can `docker push` the image to your choice of registry for deployment,
or run it locally for development and testing:

In [ ]:
!docker run -p 5000:5000 iris-classifier

## Loading BentoService in Python

`bentoml.load` is the enssential API for loading a BentoArchive into your
python application:

In [ ]:
import bentoml
import pandas as pd

bento_svc = bentoml.load('./model')

# Test loaded bentoml service:
bento_svc.predict([X[0]])

## "pip install" a BentoML archive

BentoML also supports distributing a BentoService as PyPI package, with the
generated `setup.py` file. A BentoArchive can be installed with `pip`:

In [ ]:
!pip install ./model

Now you can import your ML service as a regular python package:

In [ ]:
import IrisClassifier

installed_svc = IrisClassifier.load()
installed_svc.predict([X[0]])

With the `setup.py` config, a BentoArchive can also be uploaded to pypi.org
as a public python package, or to your organization's private PyPI index for all
developers in your organization to use:

`cd ./model & python setup.py sdist upload`

*You will need a ".pypirc" config file before doing this: https://docs.python.org/2/distutils/packageindex.html*


# CLI access

`pip install ./model` also installs a CLI tool for accessing the BentoML service:

In [ ]:
!IrisClassifier info

In [ ]:
!IrisClassifier --help

In [ ]:
!IrisClassifier predict --help

In [ ]:
!IrisClassifier predict --input='[[5.1, 3.5, 1.4, 0.2]]'

BentoML cli also supports reading input data from `csv` or `json` files, in either local machine or remote HTTP/S3 location:

In [ ]:
# Writing test data to a csv file
pd.DataFrame(iris.data).to_csv('iris_data.csv', index=False)

In [ ]:
# Invoke predict from command lien
!IrisClassifier predict --input='./iris_data.csv'

Alternatively, you can also use the `bentoml` cli to load and run a BentoML service archive without installing it:

In [ ]:
!bentoml info ./model

In [ ]:
!bentoml predict ./model --input='[[5.1, 3.5, 1.4, 0.2]]'